In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf

%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [10]:
path = "../training/conf/network/encoder/efficientnet.yaml"

In [11]:
with open(path, "rb") as f:
    cfg = OmegaConf.load(f)

In [12]:
cfg

{'_target_': 'text_recognizer.networks.encoders.efficientnet.EfficientNet', 'arch': 'b0', 'stochastic_dropout_rate': 0.2, 'bn_momentum': 0.99, 'bn_eps': 0.001}

In [13]:
cfg.arch = "b2"

In [14]:
net = instantiate(cfg)

In [15]:
from torchinfo import summary

In [16]:
summary(net, (2, 1, 576, 640), device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
EfficientNet                             --                        --
├─ModuleList: 1-1                        --                        --
├─Sequential: 1-2                        [2, 32, 288, 320]         --
│    └─ZeroPad2d: 2-1                    [2, 1, 577, 641]          --
│    └─Conv2d: 2-2                       [2, 32, 288, 320]         288
│    └─BatchNorm2d: 2-3                  [2, 32, 288, 320]         64
│    └─Mish: 2-4                         [2, 32, 288, 320]         --
├─ModuleList: 1-1                        --                        --
│    └─MBConvBlock: 2-5                  [2, 16, 288, 320]         --
│    │    └─Sequential: 3-1              [2, 32, 288, 320]         352
│    │    └─Sequential: 3-2              [2, 32, 288, 320]         552
│    │    └─Sequential: 3-3              [2, 16, 288, 320]         544
│    └─MBConvBlock: 2-6                  [2, 16, 288, 320]         --
│    │    └

In [11]:
net[:-2]

TypeError: 'EfficientNet' object is not subscriptable